# Subgraph Isomorphism

### Useful References

* [Subgraph Isomorphism Problem](https://en.wikipedia.org/wiki/Subgraph_isomorphism_problem#CITEREFUllmann2010)
* [Original Algorithm](https://dl.acm.org/doi/pdf/10.1145/321921.321925)

In [27]:
from queue import LifoQueue
from copy import deepcopy

import math as m

### All possible subgraphs

![K3inK6](./media/videos/subgraph_search/720p30/K3inK6.gif "K3 in K6")

More GIFs:
* [K4 in K6](./media/videos/subgraph_search/720p30/K4inK6.gif)
* [K4 in K8](./media/videos/subgraph_search/720p30/K4inK8.gif)
* [K9 in K10](./media/videos/subgraph_search/720p30/K9inK10.gif)

### Choosing Vertices

In [28]:
def enumerate_arrangements(n: int) -> list:
    arrangements = []
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == n:
            arrangements.append(item)
        else:
            for i in range(n):
                if not i in item:
                    new_item = deepcopy(item)
                    new_item.append(i)
                    stack.put(new_item)

    return arrangements

In [29]:
def enumerate_choices(n: int, r: int) -> list:
    choices = []
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == r:
            choices.append(item)
        else:
            for i in range((item[-1] + 1) if len(item) > 0 else 0, n):
                new_item = deepcopy(item)
                new_item.append(i)
                stack.put(new_item)

    return choices

In [34]:
n = 6
r = 3

arrangements = enumerate_arrangements(n)
choices = enumerate_choices(n, r)

print(len(arrangements), arrangements)
print(len(choices), choices)

assert(len(arrangements) == m.factorial(n))
assert(len(choices) == m.comb(n, r))

720 [[5, 4, 3, 2, 1, 0], [5, 4, 3, 2, 0, 1], [5, 4, 3, 1, 2, 0], [5, 4, 3, 1, 0, 2], [5, 4, 3, 0, 2, 1], [5, 4, 3, 0, 1, 2], [5, 4, 2, 3, 1, 0], [5, 4, 2, 3, 0, 1], [5, 4, 2, 1, 3, 0], [5, 4, 2, 1, 0, 3], [5, 4, 2, 0, 3, 1], [5, 4, 2, 0, 1, 3], [5, 4, 1, 3, 2, 0], [5, 4, 1, 3, 0, 2], [5, 4, 1, 2, 3, 0], [5, 4, 1, 2, 0, 3], [5, 4, 1, 0, 3, 2], [5, 4, 1, 0, 2, 3], [5, 4, 0, 3, 2, 1], [5, 4, 0, 3, 1, 2], [5, 4, 0, 2, 3, 1], [5, 4, 0, 2, 1, 3], [5, 4, 0, 1, 3, 2], [5, 4, 0, 1, 2, 3], [5, 3, 4, 2, 1, 0], [5, 3, 4, 2, 0, 1], [5, 3, 4, 1, 2, 0], [5, 3, 4, 1, 0, 2], [5, 3, 4, 0, 2, 1], [5, 3, 4, 0, 1, 2], [5, 3, 2, 4, 1, 0], [5, 3, 2, 4, 0, 1], [5, 3, 2, 1, 4, 0], [5, 3, 2, 1, 0, 4], [5, 3, 2, 0, 4, 1], [5, 3, 2, 0, 1, 4], [5, 3, 1, 4, 2, 0], [5, 3, 1, 4, 0, 2], [5, 3, 1, 2, 4, 0], [5, 3, 1, 2, 0, 4], [5, 3, 1, 0, 4, 2], [5, 3, 1, 0, 2, 4], [5, 3, 0, 4, 2, 1], [5, 3, 0, 4, 1, 2], [5, 3, 0, 2, 4, 1], [5, 3, 0, 2, 1, 4], [5, 3, 0, 1, 4, 2], [5, 3, 0, 1, 2, 4], [5, 2, 4, 3, 1, 0], [5, 2, 4, 3, 0,